In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/reversible///')
%cd /home/schirrmr/


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

In [ ]:
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var, var_to_np
from IPython.display import display

In [ ]:
x = np_to_var([-1.5,1.5], dtype=np.float32, requires_grad=True)
valid_x = np_to_var([-0.5, 0.5], dtype=np.float32, requires_grad=True)
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)
plt.scatter(var_to_np(x),  var_to_np(x) * 0, )
plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0, )
plt.scatter(var_to_np(x_fake),  var_to_np(x_fake) * 0 - 0.1, alpha=0.2, marker='x')

In [ ]:
from torch.nn import ConstantPad2d
import torch as th
from torch import nn

### Train ReLU discriminator 

In [ ]:


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20190425, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,1))


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(model.parameters(), lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x, valid_x)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:

        real_out = model(th.cat((x, valid_x)).unsqueeze(1))
        fake_out = model(x_fake.unsqueeze(1))
        loss_score = -(th.mean(real_out) - th.mean(fake_out))
        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

## minibatch of size 3

In [ ]:


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20190425, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,1))


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(model.parameters(), lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 20000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x, valid_x))[th.randperm(4)[:3]].unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        real_out = model(th.cat((x, valid_x)).unsqueeze(1))
        fake_out = model(x_fake.unsqueeze(1))
        loss_score = -(th.mean(real_out) - th.mean(fake_out))
        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

-> needs more epochs but still works

### minibatch of size 1?

In [ ]:


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20190425, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,1))


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(model.parameters(), lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 80000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x, valid_x))[th.randperm(4)[:1]].unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        real_out = model(th.cat((x, valid_x)).unsqueeze(1))
        fake_out = model(x_fake.unsqueeze(1))
        loss_score = -(th.mean(real_out) - th.mean(fake_out))
        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

## train weight norm on valid

In [ ]:
fake_out.shape

In [ ]:
from reversible2.weight_norm import weight_norm
import itertools


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(201904251, False) # worked with 102398213
norms = [th.ones((40,1), requires_grad=True), th.ones((40,1), requires_grad=True),
         th.ones((1,1), requires_grad=True)]
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,40), nn.ReLU(), 
                      nn.Linear(40,1))
weight_norm(model[0], fixed_norm=norms[0])
weight_norm(model[2], fixed_norm=norms[1])
weight_norm(model[4], fixed_norm=norms[2])


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(itertools.chain(model.parameters(), norms), lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x, valid_x)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

### with fixed norms

In [ ]:
from reversible2.weight_norm import weight_norm
import itertools


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(201904251, False) # worked with 102398213
norms = [th.ones((40,1), requires_grad=True), th.ones((40,1), requires_grad=True),
         th.ones((1,1), requires_grad=True)]
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,40), nn.ReLU(), 
                      nn.Linear(40,1))
weight_norm(model[0], fixed_norm=norms[0])
weight_norm(model[2], fixed_norm=norms[1])
weight_norm(model[4], fixed_norm=norms[2])


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(itertools.chain(model.parameters(), ), lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x, valid_x)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss.item()))
        

### optimizing norm on valid

In [ ]:
from reversible2.weight_norm import weight_norm
import itertools


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(201904251, False) # worked with 102398213
norms = [th.ones((40,1), requires_grad=True), th.ones((40,1), requires_grad=True),
         th.ones((1,1), requires_grad=True)]
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,40), nn.ReLU(), 
                      nn.Linear(40,1))
weight_norm(model[0], fixed_norm=norms[0])
weight_norm(model[2], fixed_norm=norms[1])
weight_norm(model[4], fixed_norm=norms[2])


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(itertools.chain(model.parameters(), ), lr=5e-3)
optim_wnorm = th.optim.Adam(norms, lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 20000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x,)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    valid_out = model(valid_x.unsqueeze(1))
    loss = -th.mean(valid_out)
    optim_wnorm.zero_grad()
    loss.backward()
    optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

In [ ]:
from reversible2.weight_norm import weight_norm
import itertools


from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(201904251, False) # worked with 102398213
norms = [th.zeros((40,1), requires_grad=True), th.zeros((40,1), requires_grad=True),
         th.zeros((1,1), requires_grad=True)]
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,40), nn.ReLU(), 
                      nn.Linear(40,1))
weight_norm(model[0], fixed_log_norm=norms[0])
weight_norm(model[2], fixed_log_norm=norms[1])
weight_norm(model[4], fixed_log_norm=norms[2])


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(itertools.chain(model.parameters(), ), lr=5e-3)
optim_wnorm = th.optim.Adam(norms[1:], lr=5e-3)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 20000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x,)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))
    valid_out = model(th.cat((x, valid_x)).unsqueeze(1))
    loss = -th.mean(valid_out) + th.mean(fake_out) + loss_grad * 100
    optim_wnorm.zero_grad()
    loss.backward()
    optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:

        real_out = model(th.cat((x, valid_x)).unsqueeze(1))
        fake_out = model(x_fake.unsqueeze(1))
        loss_score = -(th.mean(real_out) - th.mean(fake_out))
        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

In [ ]:
[th.exp(n) for n in norms]

## optimizer weights but not bias on valid?

In [ ]:
from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(201904251, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,40), nn.ReLU(), 
                      nn.Linear(40,1))


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))

optimizer = th.optim.Adam(model.parameters(), lr=5e-3)
weights = [m.weight for m in model.modules() if hasattr(m, 'weight')]
optim_wnorm = th.optim.Adam(weights, lr=1e-2)

In [ ]:
from reversible2.gradient_penalty import gradient_penalty

x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x,)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss_score = -(th.mean(real_out) - th.mean(fake_out))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = loss_score + loss_grad * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    valid_out = model(valid_x.unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))
    loss_grad = gradient_penalty(model, th.cat((x, valid_x)).repeat(100).unsqueeze(1), x_fake.unsqueeze(1))
    loss = -th.mean(valid_out) + th.mean(fake_out) + loss_grad * 100
    optim_wnorm.zero_grad()
    loss.backward()
    optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.scatter(var_to_np(x),  var_to_np(x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0 + var_to_np(th.mean(fake_out)), )
        plt.legend(("Score", "Training", "Validation"), bbox_to_anchor=(1,1,0,0))
        display(fig)
        plt.close(fig)
        print("Loss {:.1E}".format(loss_score.item()))
        

In [ ]:
loss_grad